In [ ]:
# pip install yahooquery

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import yahooquery as yq
import yfinance as yf

In [ ]:
from functools import reduce

## Creation of S&P 500 Constituents Company List

In [ ]:
# Read and print the stock tickers that make up S&P500
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

columns_to_remove = ["Headquarters Location", "Date added", "CIK", "Founded"]
tickers = tickers.drop(columns=columns_to_remove)

tickers = tickers.rename(columns={"Security": "Full Name"})

tickers

,Symbol,Full Name,GICS Sector,GICS Sub-Industry
0,MMM,3M,Industrials,Industrial Conglomerates
1,AOS,A. O. Smith,Industrials,Building Products
2,ABT,Abbott,Health Care,Health Care Equipment
3,ABBV,AbbVie,Health Care,Pharmaceuticals
4,ACN,Accenture,Information Technology,IT Consulting & Other Services
...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment
501,ZION,Zions Bancorporation,Financials,Regional Banks


In [ ]:
# Iterate through each symbol in the DataFrame
yq_data = pd.DataFrame([])

for ticker in tickers['Symbol']:
    try:

      # Use yahooquery to fetch ESG data
      ticker_yq = yq.Ticker(ticker).esg_scores

      ticker_yq[ticker]["Symbol"] = ticker

      ticker_yq = pd.DataFrame([ticker_yq[ticker]])
      yq_data = pd.concat([yq_data, ticker_yq], ignore_index=True)

    except Exception as e:
      # print(e)
      continue

yq_data

,maxAge,totalEsg,environmentScore,socialScore,governanceScore,ratingYear,ratingMonth,highestControversy,peerCount,esgPerformance,...,furLeather,gambling,gmo,militaryContract,nuclear,pesticides,palmOil,coal,tobacco,Symbol
0,86400,33.61,12.33,13.56,7.72,2022,8,3.0,34,OUT_PERF,...,False,False,False,False,False,False,False,False,False,MMM
1,86400,25.43,7.29,11.87,6.27,2023,1,0.0,44,AVG_PERF,...,False,False,False,False,False,False,False,False,False,AOS
2,86400,24.98,3.03,13.59,8.36,2022,8,3.0,58,AVG_PERF,...,False,False,False,False,False,False,False,False,False,ABT
3,86400,27.84,1.12,16.77,9.95,2022,8,3.0,82,AVG_PERF,...,False,False,False,False,False,False,False,False,False,ABBV
4,86400,9.71,0.29,4.58,4.83,2022,8,2.0,105,LAG_PERF,...,False,False,False,False,False,False,False,False,False,ACN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,86400,23.80,12.87,6.75,4.18,2023,1,2.0,91,AVG_PERF,...,False,False,False,False,False,False,False,False,False,XEL
428,86400,16.04,3.56,7.40,5.08,2023,1,1.0,44,UNDER_PERF,...,False,False,False,False,False,False,False,False,False,XYL
429,86400,20.55,4.46,11.65,4.44,2022,8,2.0,44,AVG_PERF,...,False,False,False,False,False,False,False,False,False,YUM
430,86400,27.30,4.46,14.60,8.24,2023,1,2.0,37,AVG_PERF,...,False,False,False,False,False,False,False,False,False,ZBH


In [ ]:
required_cols = ["Symbol", "environmentScore", "socialScore", "governanceScore", "totalEsg", "highestControversy", "percentile"]

yq_data.columns.name = ''
yq_data = yq_data[required_cols]
yq_data = yq_data.apply(pd.to_numeric, errors="ignore")

In [ ]:
yq_data = yq_data.rename(columns={"environmentScore": "Environment Score", "socialScore": "Social Score", "governanceScore": "Governance Score", "totalEsg": "Total Esg", "highestControversy": "Highest Controversy", "percentile": "Percentile"})

In [ ]:
yq_data

,Symbol,Environment Score,Social Score,Governance Score,Total Esg,Highest Controversy,Percentile
0,MMM,12.33,13.56,7.72,33.61,3.0,74.68
1,AOS,7.29,11.87,6.27,25.43,0.0,46.64
2,ABT,3.03,13.59,8.36,24.98,3.0,44.01
3,ABBV,1.12,16.77,9.95,27.84,3.0,55.08
4,ACN,0.29,4.58,4.83,9.71,2.0,2.21
...,...,...,...,...,...,...,...
427,XEL,12.87,6.75,4.18,23.80,2.0,40.01
428,XYL,3.56,7.40,5.08,16.04,1.0,12.91
429,YUM,4.46,11.65,4.44,20.55,2.0,27.31
430,ZBH,4.46,14.60,8.24,27.30,2.0,54.32


In [ ]:
# Iterate through each symbol in the DataFrame
yf_data = pd.DataFrame([])

for ticker in tickers['Symbol']:
    try:

      # Use yfinance to fetch data for each symbol
      ticker_info = yf.Ticker(ticker).info

      # Create a DataFrame for the current symbol's data
      ticker_data = pd.DataFrame({
        'Symbol': [ticker],
        'Market Cap': [ticker_info.get('marketCap')],
        'Beta': [ticker_info.get('beta')],
        'Overall Risk': [ticker_info.get('overallRisk')]
      })

      yf_data = pd.concat([yf_data, ticker_data], ignore_index=True)

    except Exception as e:
      # print(e)
      continue

yf_data

,Symbol,Market Cap,Beta,Overall Risk
0,MMM,55784308736,0.968188,7
1,AOS,10048283648,1.286998,10
2,ABT,176555524096,0.673236,8
3,ABBV,268499402752,0.547026,9
4,ACN,197301125120,1.215421,1
...,...,...,...,...
498,YUM,35864207360,1.010373,3
499,ZBRA,12905447424,1.668049,8
500,ZBH,25349423104,1.035269,6
501,ZION,5469512704,1.141078,6


In [ ]:
output = reduce(lambda x,y: pd.merge(x,y, on='Symbol', how='outer'), [tickers, yq_data, yf_data])

In [ ]:
output

,Symbol,Full Name,GICS Sector,GICS Sub-Industry,Environment Score,Social Score,Governance Score,Total Esg,Highest Controversy,Percentile,Market Cap,Beta,Overall Risk
0,MMM,3M,Industrials,Industrial Conglomerates,12.33,13.56,7.72,33.61,3.0,74.68,55784308736,0.968188,7
1,AOS,A. O. Smith,Industrials,Building Products,7.29,11.87,6.27,25.43,0.0,46.64,10048283648,1.286998,10
2,ABT,Abbott,Health Care,Health Care Equipment,3.03,13.59,8.36,24.98,3.0,44.01,176555524096,0.673236,8
3,ABBV,AbbVie,Health Care,Pharmaceuticals,1.12,16.77,9.95,27.84,3.0,55.08,268499402752,0.547026,9
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,0.29,4.58,4.83,9.71,2.0,2.21,197301125120,1.215421,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,4.46,11.65,4.44,20.55,2.0,27.31,35864207360,1.010373,3
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,NaN,NaN,NaN,NaN,NaN,NaN,12905447424,1.668049,8
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,4.46,14.60,8.24,27.30,2.0,54.32,25349423104,1.035269,6
501,ZION,Zions Bancorporation,Financials,Regional Banks,NaN,NaN,NaN,NaN,NaN,NaN,5469512704,1.141078,6


## ML - Data Preprocessing

In [ ]:
output.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 503 entries, 0 to 502
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Symbol               503 non-null    object 
 1   Full Name            503 non-null    object 
 2   GICS Sector          503 non-null    object 
 3   GICS Sub-Industry    503 non-null    object 
 4   Environment Score    432 non-null    float64
 5   Social Score         432 non-null    float64
 6   Governance Score     432 non-null    float64
 7   Total Esg            432 non-null    float64
 8   Highest Controversy  432 non-null    float64
 9   Percentile           432 non-null    float64
 10  Market Cap           501 non-null    object 
 11  Beta                 494 non-null    object 
 12  Overall Risk         498 non-null    object 
dtypes: float64(6), object(7)
memory usage: 55.0+ KB


In [ ]:
output.isnull().sum()

Symbol                  0
Full Name               0
GICS Sector             0
GICS Sub-Industry       0
Environment Score      71
Social Score           71
Governance Score       71
Total Esg              71
Highest Controversy    71
Percentile             71
Market Cap              2
Beta                    9
Overall Risk            5
dtype: int64

In [ ]:
median_environment_score = output['Environment Score'].median()
median_social_score = output['Social Score'].median()
median_governance_score = output['Governance Score'].median()
median_total = ((median_environment_score + median_social_score + median_governance_score) / 3)

median_highest_controversy = output['Highest Controversy'].median()
median_percentile = output['Percentile'].median()

median_market_cap = output['Market Cap'].median()
median_beta = output['Beta'].median()
median_overall_risk = output['Overall Risk'].median()

In [ ]:
# Fill missing values in each column with the median of that column
output['Environment Score'].fillna(median_environment_score, inplace=True)
output['Social Score'].fillna(median_social_score, inplace=True)
output['Governance Score'].fillna(median_governance_score, inplace=True)
output['Total Esg'].fillna(median_total, inplace=True)
output['Highest Controversy'].fillna(median_highest_controversy, inplace=True)
output['Percentile'].fillna(median_percentile, inplace=True)
output['Market Cap'].fillna(median_market_cap, inplace=True)
output['Beta'].fillna(median_beta, inplace=True)
output['Overall Risk'].fillna(median_overall_risk, inplace=True)

In [ ]:
output['Overall Risk'] = output['Overall Risk'].astype(int)
output['Highest Controversy'] = output['Highest Controversy'].astype(int)

In [ ]:
output.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 503 entries, 0 to 502
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Symbol               503 non-null    object 
 1   Full Name            503 non-null    object 
 2   GICS Sector          503 non-null    object 
 3   GICS Sub-Industry    503 non-null    object 
 4   Environment Score    503 non-null    float64
 5   Social Score         503 non-null    float64
 6   Governance Score     503 non-null    float64
 7   Total Esg            503 non-null    float64
 8   Highest Controversy  503 non-null    int64  
 9   Percentile           503 non-null    float64
 10  Market Cap           503 non-null    float64
 11  Beta                 503 non-null    float64
 12  Overall Risk         503 non-null    int64  
dtypes: float64(7), int64(2), object(4)
memory usage: 55.0+ KB


In [ ]:
output

,Symbol,Full Name,GICS Sector,GICS Sub-Industry,Environment Score,Social Score,Governance Score,Total Esg,Highest Controversy,Percentile,Market Cap,Beta,Overall Risk
0,MMM,3M,Industrials,Industrial Conglomerates,12.33,13.56,7.72,33.61,3,74.68,5.578431e+10,0.968188,7
1,AOS,A. O. Smith,Industrials,Building Products,7.29,11.87,6.27,25.43,0,46.64,1.004828e+10,1.286998,10
2,ABT,Abbott,Health Care,Health Care Equipment,3.03,13.59,8.36,24.98,3,44.01,1.765555e+11,0.673236,8
3,ABBV,AbbVie,Health Care,Pharmaceuticals,1.12,16.77,9.95,27.84,3,55.08,2.684994e+11,0.547026,9
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,0.29,4.58,4.83,9.71,2,2.21,1.973011e+11,1.215421,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,4.46,11.65,4.44,20.55,2,27.31,3.586421e+10,1.010373,3
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,3.70,8.73,6.20,6.21,2,29.85,1.290545e+10,1.668049,8
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,4.46,14.60,8.24,27.30,2,54.32,2.534942e+10,1.035269,6
501,ZION,Zions Bancorporation,Financials,Regional Banks,3.70,8.73,6.20,6.21,2,29.85,5.469513e+09,1.141078,6


In [ ]:
output.to_csv('./outputs_csv/s&p500_esg_data.csv', index=False)

## Creation of S&P 500 Constituents Price List

In [ ]:
start_date = "2015-01-01"
end_date = "2022-12-31"

In [ ]:
ticker_name = tickers["Symbol"].to_list()

In [ ]:
price_data = yf.download(ticker_name, start=start_date, end=end_date)

[*********************100%%**********************]  503 of 503 completed

ERROR:yfinance:
3 Failed downloads:
ERROR:yfinance:['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
ERROR:yfinance:['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2015-01-01 -> 2022-12-31)')
ERROR:yfinance:['KVUE']: Exception("%ticker%: Data doesn't exist for startDate = 1420088400, endDate = 1672462800")


In [ ]:
prices = price_data["Adj Close"][ticker_name]

In [ ]:
prices.head()

,MMM,AOS,ABT,ABBV,ACN,ATVI,ADM,ADBE,ADP,AES,...,WTW,GWW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02 00:00:00,123.082413,24.458412,38.058811,45.607620,76.840805,18.852276,40.796497,72.339996,68.902222,10.195618,...,102.877754,217.541626,129.002014,27.616508,34.026726,44.099751,77.430000,102.223495,23.135227,40.775261
2015-01-05 00:00:00,120.306541,24.030470,38.067291,44.749317,75.543396,18.590050,39.384857,71.980003,68.852600,9.905375,...,102.138794,212.222748,126.806435,27.303040,31.908997,43.203735,76.339996,106.036827,22.268372,40.530472
2015-01-06 00:00:00,119.023659,23.833960,37.634998,44.527832,74.998474,18.243534,38.608452,70.529999,68.281792,9.682117,...,101.630768,209.213928,124.101494,27.448309,31.721346,42.673447,75.790001,105.137962,21.417871,40.135044
2015-01-07 00:00:00,119.886414,24.008633,37.940136,46.327477,76.572685,17.850193,39.188793,71.110001,68.877434,9.696998,...,103.085571,209.401459,127.746109,27.685318,31.971550,44.087570,77.720001,107.752792,21.622320,40.963543
2015-01-08 00:00:00,122.759804,24.633085,38.719971,46.812008,77.740326,18.028135,38.616287,72.919998,70.457474,9.845840,...,104.263298,214.063889,129.282990,28.006453,32.212811,44.855572,79.379997,108.896782,21.924900,41.594334


In [ ]:
prices.to_csv('./outputs_csv/s&p500_price_data.csv')